In [1]:
import pandas as pd
import os
from glob import glob

# Verzeichnis mit den Precipitation-CSV-Dateien
precip_dir = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/Niederschlag/Saisonal"
output_dir = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/Niederschlag/Saisonal/processed/"

# Ausgabe-Verzeichnis (wird automatisch angelegt, falls es nicht existiert)
output_dir = os.path.join(output_dir, "by_year")
os.makedirs(output_dir, exist_ok=True)

# Für jede CSV-Datei im Verzeichnis
for csv_file in glob(os.path.join(precip_dir, "*.csv")):
    # Einlesen
    df = pd.read_csv(csv_file, dtype=str)
    
    # Gemeinsame Basis-Spalten behalten
    common_cols = ['ID', 'ARS', 'AGS', 'GEN', 'BEZ']
    
    # Alle Precipitation-Spalten identifizieren
    precip_cols = [c for c in df.columns if c.startswith('precipitation_')]
    
    # Jahre extrahieren (erste 4 Zeichen nach 'precipitation_')
    years = sorted({c.split('_')[1][:4] for c in precip_cols})
    
    # Pro Jahr eine eigene CSV-Datei erstellen
    for year in years:
        # Spalten für dieses Jahr auswählen
        year_cols = [c for c in precip_cols if c.split('_')[1].startswith(year)]
        # Umbenennung: '_YYYY' entfernen, sodass z.B. 'precipitation_202213_mean' zu 'precipitation_13_mean' wird
        rename_map = {
            c: c.replace(f"_{year}", "")
            for c in year_cols
        }
        # DataFrame für dieses Jahr zusammenstellen
        df_year = df[common_cols + year_cols].rename(columns=rename_map)
        
        # Ausgabepfad bauen
        base_name = os.path.splitext(os.path.basename(csv_file))[0]
        out_file = os.path.join(output_dir, f"{base_name}_{year}.csv")
        
        # Speichern
        df_year.to_csv(out_file, index=False)

print("✅ Alle Dateien wurden nach Jahr aufgeteilt und gespeichert.")


✅ Alle Dateien wurden nach Jahr aufgeteilt und gespeichert.


In [4]:
import pandas as pd
import os
from glob import glob

# Ordner mit den CSV-Dateien, die umbenannt werden sollen
input_dir = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/Niederschlag/Saisonal/processed/by_year"

output_dir = os.path.join(input_dir, "renamed")
os.makedirs(output_dir, exist_ok=True)

# Mapping von Nummern zu Jahreszeiten
season_map = {
    '13': 'MAM',
    '14': 'JJA',
    '15': 'SON',
    '16': 'DJF'
}

# Alle CSV-Dateien im Input-Verzeichnis verarbeiten
for file in glob(os.path.join(input_dir, "*.csv")):
    df = pd.read_csv(file, dtype=str)
    
    # Spalten umbenennen
    rename_dict = {}
    for col in df.columns:
        # Prüfen, ob Spalte mit precipitation beginnt
        if col.startswith('precipitation'):
            # Extrahiere Nummer und Suffix (_mean/_median)
            parts = col.split('_')
            num = parts[0].replace('precipitation', '')
            suffix = parts[1] if len(parts) > 1 else ''
            if num in season_map and suffix in ['mean', 'median']:
                new_col = f"Niederschlag_{season_map[num]}_{suffix}"
                rename_dict[col] = new_col
    
    df.rename(columns=rename_dict, inplace=True)
    
    # Speichern unter neuem Namen
    base = os.path.basename(file)
    out_path = os.path.join(output_dir, base)
    df.to_csv(out_path, index=False)

print("✅ Alle Dateien wurden umbenannt und im 'renamed'-Ordner gespeichert.")


✅ Alle Dateien wurden umbenannt und im 'renamed'-Ordner gespeichert.


In [10]:
import pandas as pd
import os
from glob import glob

# Verzeichnisse
temp_dir = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/Temperatur/Saisonal/"
output_dir = os.path.join(temp_dir, "by_year_renamed")
os.makedirs(output_dir, exist_ok=True)

# Quartalscode → Jahreszeit
season_map = {'13': 'MAM', '14': 'JJA', '15': 'SON', '16': 'DJF'}

# Fixe Basis-Spalten
base_cols = ['ID', 'ARS', 'AGS', 'GEN', 'BEZ']

# Alle CSV-Dateien verarbeiten
for file in glob(os.path.join(temp_dir, "*.csv")):
    print(f"\n=== Verarbeite Datei: {os.path.basename(file)} ===")
    df = pd.read_csv(file, dtype=str)
    print("Original DF Kopf:")
    print(df.head(), "\n")
    
    # Alle Temperatur-Spalten erkennen
    temp_cols = [c for c in df.columns if c.startswith('air_temp_')]
    print("Gefundene Temperatur-Spalten:")
    print(temp_cols, "\n")
    
    # Alle Jahre in diesen Spalten sammeln
    years = sorted({c.split('_')[3][:4] for c in temp_cols})
    print("Erkannte Jahre:", years, "\n")
    
    for year in years:
        print(f"--- Bearbeite Jahr: {year} ---")
        # Spalten des aktuellen Jahres
        cols_for_year = [c for c in temp_cols if c.split('_')[3].startswith(year)]
        print("Spalten für dieses Jahr:")
        print(cols_for_year)
        
        # Rename-Map bauen
        rename_map = {}
        for col in cols_for_year:
            _, _, metric, period, stat = col.split('_')
            season = season_map.get(period[4:], period[4:])
            new_col = f"air_temp_{metric}_{season}_{stat}"
            rename_map[col] = new_col
        print("Rename-Map:")
        print(rename_map)
        
        # DataFrame für dieses Jahr
        df_year = df[base_cols + cols_for_year].rename(columns=rename_map)
        print("DF für Jahr, Kopf:")
        print(df_year.head(), "\n")
        
        # Speichern
        base = os.path.splitext(os.path.basename(file))[0]
        out = os.path.join(output_dir, f"{base}_{year}.csv")
        df_year.to_csv(out, index=False, encoding='utf-8')
        print(f"→ Gespeichert als: {os.path.basename(out)}\n")

print("✅ Fertig: Alle Temperaturdaten pro Jahr getrennt und umbenannt.") 



=== Verarbeite Datei: Stats_Temp_Saisonal.csv ===
Original DF Kopf:
  ID    ARS    AGS           GEN               BEZ air_temp_max_202213_mean  \
0  1  01001  01001     Flensburg  Kreisfreie Stadt         12.9145833333333   
1  2  01002  01002          Kiel  Kreisfreie Stadt         13.0234234234234   
2  3  01003  01003        Lübeck  Kreisfreie Stadt         13.6105263157895   
3  4  01004  01004    Neumünster  Kreisfreie Stadt         13.6528571428571   
4  5  01051  01051  Dithmarschen             Kreis         13.3796038151137   

  air_temp_max_202213_median air_temp_max_202313_mean  \
0                       12.9         12.1083333333333   
1                         13         12.3009009009009   
2                       13.7         12.9384210526316   
3                       13.6         12.8785714285714   
4                       13.3         12.6782831988261   

  air_temp_max_202313_median air_temp_max_202214_mean  ...  \
0                       12.1         22.25833333333

In [12]:
import pandas as pd
import os
from glob import glob

# Verzeichnisse
temp_dir = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/Temperatur/Saisonal/"
output_dir = os.path.join(temp_dir, "by_year_metric_de")
os.makedirs(output_dir, exist_ok=True)

# Jahreszeit-Kürzel
season_map = {'13': 'MAM', '14': 'JJA', '15': 'SON', '16': 'DJF'}

# Spaltenübersetzungen (Deutsch)
column_rename_base = {
    'ID': 'ID',
    'ARS': 'ARS',
    'AGS': 'AGS',
    'GEN': 'Gemeinde',
    'BEZ': 'Bezirk'
}

stat_map = {'mean': 'mean', 'median': 'median'}

# CSV-Dateien durchgehen
for file in glob(os.path.join(temp_dir, "*.csv")):
    print(f"\n=== Verarbeite Datei: {os.path.basename(file)} ===")
    df = pd.read_csv(file, dtype=str)

    # Temperatur-Spalten erkennen
    temp_cols = [c for c in df.columns if c.startswith('air_temp_')]

    # Verfügbare Jahre erkennen
    years = sorted({c.split('_')[3][:4] for c in temp_cols})

    for year in years:
        print(f"--- Jahr: {year} ---")

        # Jahresbezogene Spalten
        cols_for_year = [c for c in temp_cols if c.split('_')[3].startswith(year)]

        for metric in ['min', 'mean', 'max']:
            cols_for_metric = [c for c in cols_for_year if f"air_temp_{metric}_" in c]
            if not cols_for_metric:
                continue

            # Spalten umbenennen
            rename_map = {}
            for col in cols_for_metric:
                _, _, _, period, stat = col.split('_')
                season = season_map.get(period[4:], period[4:])
                new_stat = stat_map.get(stat, stat)
                new_col = f"Lufttemperatur_{metric}_{season}_{new_stat}"
                rename_map[col] = new_col

            # DataFrame erstellen
            df_metric = df[column_rename_base.keys()].rename(columns=column_rename_base)
            df_metric = pd.concat([df_metric, df[cols_for_metric].rename(columns=rename_map)], axis=1)

            # Dateiname
            out_name = f"Lufttemperatur_{metric}_{year}.csv"
            out_path = os.path.join(output_dir, out_name)

            # Speichern
            df_metric.to_csv(out_path, index=False, encoding='utf-8')
            print(f"✔ Gespeichert: {out_name}")

print("\n✅ Fertig: Lufttemperaturdaten pro Jahr und Metrik gespeichert.")



=== Verarbeite Datei: Stats_Temp_Saisonal.csv ===
--- Jahr: 2022 ---
✔ Gespeichert: Lufttemperatur_min_2022.csv
✔ Gespeichert: Lufttemperatur_mean_2022.csv
✔ Gespeichert: Lufttemperatur_max_2022.csv
--- Jahr: 2023 ---
✔ Gespeichert: Lufttemperatur_min_2023.csv
✔ Gespeichert: Lufttemperatur_mean_2023.csv
✔ Gespeichert: Lufttemperatur_max_2023.csv

✅ Fertig: Lufttemperaturdaten pro Jahr und Metrik gespeichert.


In [7]:
import pandas as pd
import os
from glob import glob

# Verzeichnisse
input_path = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/UHI/Tropische_Naechte/Saisonal/"
output_path = os.path.join(input_path, "by_year")
os.makedirs(output_path, exist_ok=True)

# Basis-Spalten (nicht umbenennen)
base_cols = ['ID', 'ARS', 'AGS', 'GEN', 'BEZ']

# Dateien einlesen
for file in glob(os.path.join(input_path, "*.csv")):
    print(f"📂 Verarbeite Datei: {os.path.basename(file)}")
    df = pd.read_csv(file, dtype=str)

    # Tropennacht-Spalten filtern
    tropen_cols = [col for col in df.columns if "tropical-nights_diff" in col]

    # Jahr → {alte_spalte: neue_spalte}
    year_map = {}
    for col in tropen_cols:
        # Datum suchen (Teil mit YYYY-MM-DD)
        parts = col.split("_")
        date_parts = [p for p in parts if p.startswith("20")]
        if not date_parts:
            continue
        year, month, *_ = date_parts[0].split("-")
        stat = parts[-1]  # mean, median oder max
        new_col = f"Tropennaechte_{month}_{stat}"
        year_map.setdefault(year, {})[col] = new_col

    # Für jedes Jahr ein CSV speichern
    for year, rename_dict in year_map.items():
        cols = base_cols + list(rename_dict.keys())
        df_year = df[cols].rename(columns=rename_dict)

        out_file = os.path.join(output_path, f"Tropennaechte_{year}.csv")
        df_year.to_csv(out_file, index=False, encoding='utf-8')
        print(f"✅ Gespeichert: Tropennaechte_{year}.csv")

print("\n🏁 Fertig: Tropennächte pro Jahr exportiert.")


📂 Verarbeite Datei: Stats_Tropical_Nights_Saisonal.csv
✅ Gespeichert: Tropennaechte_2022.csv
✅ Gespeichert: Tropennaechte_2023.csv
✅ Gespeichert: Tropennaechte_2024.csv

🏁 Fertig: Tropennächte pro Jahr exportiert.


In [9]:
import pandas as pd
import os
from glob import glob

# Eingabepfad & Ausgabeordner
input_path = "C:/Users/agnes/Documents/EAGLE/Innovation_Lab/Daten/DWDdaten/Kreisebene/UHI/Tropische_Naechte/Jaehrlich/"  # <-- anpassen
output_path = os.path.join(input_path, "by_year")
os.makedirs(output_path, exist_ok=True)

# Basis-Spalten
base_cols = ['ID', 'ARS', 'AGS', 'GEN', 'BEZ']

# CSV-Dateien durchgehen
for file in glob(os.path.join(input_path, "*.csv")):
    print(f"📂 Verarbeite Datei: {os.path.basename(file)}")
    df = pd.read_csv(file, dtype=str)

    # Relevante Tropennacht-Spalten
    tropen_cols = [col for col in df.columns if "tropical-nights_diff" in col]

    # Jahr → {alte_spalte: neue_spalte}
    year_map = {}
    for col in tropen_cols:
        # Datum suchen
        parts = col.split("_")
        date_parts = [p for p in parts if p.startswith("20")]
        if not date_parts:
            continue
        year = date_parts[0].split("-")[0]
        stat = parts[-1]  # mean / median / max
        new_col = f"Tropennaechte_{stat}"
        year_map.setdefault(year, {})[col] = new_col

    # Pro Jahr speichern
    for year, rename_dict in year_map.items():
        cols = base_cols + list(rename_dict.keys())
        df_year = df[cols].rename(columns=rename_dict)

        out_file = os.path.join(output_path, f"Tropennaechte_{year}_Jahresdaten.csv")
        df_year.to_csv(out_file, index=False, encoding='utf-8')
        print(f"✅ Gespeichert: Tropennaechte_{year}_Jahresdaten.csv")

print("\n🏁 Fertig: Jahresdaten pro Jahr exportiert.")


📂 Verarbeite Datei: Stats_Tropical_Nights_Jaehrlich.csv
✅ Gespeichert: Tropennaechte_2022_Jahresdaten.csv
✅ Gespeichert: Tropennaechte_2023_Jahresdaten.csv
✅ Gespeichert: Tropennaechte_2024_Jahresdaten.csv

🏁 Fertig: Jahresdaten pro Jahr exportiert.
